In [1]:
# Combining multiple datasets to see if US counties 2016 unemployment 
# rates and minimum wage data was associated with 2016 preseidential election votes

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

unemp_county = pd.read_csv('us_unempoly_county.csv') # importing unemployment data

unemp_county.head()

df = pd.read_csv('minwage') # opening the minimum wage dataset

min_wage_data = pd.DataFrame() # saving the imported data into a new variable

for name, group in df.groupby ('State'): # Grouping by state 
    if min_wage_data.empty:
        min_wage_data = group.set_index("Year")[["Low.2018"]].rename(columns={"Low.2018":name}) # sorting the data by the lowest min wage value for that respective year
    else:
        min_wage_data = min_wage_data.join(group.set_index("Year")[["Low.2018"]].rename(columns={"Low.2018":name}))
        
min_wage_data = min_wage_data.replace(0, np.NaN).dropna(axis=1) # replace missing values replaced with NaN

min_wage_data.head()

# will now attempt to create a new column within the unemployment data with the minimum wage values (the slow way using functions and mapping)

def get_min_wage(year, state):
    try:
        return min_wage_data.loc[year][state]
    except:
        return np.NaN
    
get_min_wage(2012, "Colorado")

unemp_county['min_wage'] = list(map(get_min_wage, unemp_county['Year'], unemp_county['State'])) # mapping the function of get min wage into a column taking the parameters of unemp_county's state and year data

#unemp_county.head() # top 5 rows of the data

#unemp_county.tail() # bottom 5 rows of the data



In [2]:
unemp_county[['min_wage','Rate']].corr() # want to see if there is a correlation between the minimum wage of a state and unemployment rate
# Slightly positive correlation between minimum wage values and unemployment values, but positive nevertheless



,min_wage,Rate
min_wage,1.000000,0.153047
Rate,0.153047,1.000000


In [3]:
unemp_county[['min_wage','Rate']].cov() # want to see how much variance is shared between the minimum wage and unemployment rate

# slight relationship between minimum wage and unemployment rate of 0.65, could have some other factors that are
# playing a part in the how these two vary together

# basic matplotlib practice 

# x = unemp_county['min_wage']
# y = unemp_county['Rate']
# plt.xlabel('Minimum Wage')
# plt.ylabel('Unemployment Rate')
# plt.scatter(x,y,c='g', label= 'Values')
# plt.legend(loc= 'Upper Left')
# plt.show()

,min_wage,Rate
min_wage,1.874228,0.651586
Rate,0.651586,9.687873


In [4]:
# Time to see if presidential election results are linked with minimum wage data and unemployment rates

pres16 = pd.read_csv('pres16results.csv')

pres16.head(10) # Will just be using the column 'pct' to map correlation values with min wage and unemployment rates

len(unemp_county) # 885,548 values! wayy to much so wil shall condense it and gather only the latest data, 'feb 16'

#len(unemp_county['State'].unique()) # only 47 states listed in this dataframe

county_2016 = unemp_county.copy()[(unemp_county['Year']==2016) &(unemp_county['Month']== 'February')]

#len(county_2016) # went from 885,548 values to 2821!

#len(county_2016['State'].unique()) # 46 states when data was streamlined, one state didn't have data for feb 2016?

pres16['st'].unique() # states are listed as the postal code and not the full name, so have to convert this
# to be compatiable with they way the states are listed in county_2016

state_abrs = pd.read_csv('state_abrs.csv', index_col=0) #luckilly i have a csv file that already has these information from a previous example!

state_abrs.head()

state_abrs = state_abrs[['Postal Code']] # double brackets!!

state_abrs.head()

state_abrs_dict = state_abrs.to_dict()['Postal Code'] # creating a dictionary of postal codes that will be used to
# map onto the columns from the pres16 data, from postal code to actual state name

state_abrs_dict

county_2016['State'] = county_2016['State'].map(state_abrs_dict) # mapping the state abrs dictionary onto 
# the state column of county_2016


In [5]:
county_2016.tail() # looks like it worked!

,Year,Month,State,County,Rate,min_wage
70740,2016,February,ME,Somerset County,6.6,7.82
70741,2016,February,ME,Oxford County,5.2,7.82
70742,2016,February,ME,Knox County,4.7,7.82
70743,2016,February,ME,Piscataquis County,5.7,7.82
70744,2016,February,ME,Aroostook County,5.9,7.82


In [6]:
print (len(county_2016)) 
print (len(pres16)) # longer value than county 2016

2821
18475


In [7]:
pres16.columns # pres16 and county 2016 don't have the same column names, we have to fix that


Index(['county', 'fips', 'cand', 'st', 'pct_report', 'votes', 'total_votes',
       'pct', 'lead'],
      dtype='object')

In [8]:
county_2016.columns

Index(['Year', 'Month', 'State', 'County', 'Rate', 'min_wage'], dtype='object')

In [9]:
pres16.rename(columns={'county':'County', 'st':'State'}, inplace=True) # renaming the pres16 column to match county_2016
pres16.tail()



,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
18470,Yauco Municipio,72153,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18471,Midway Islands,74300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18472,St. Croix Island,78010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18473,St. John Island,78020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18474,St. Thomas Island,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for df in [county_2016, pres16]:
    df.set_index(["County", "State"], inplace=True)
    
pres16 = pres16[pres16['cand']=="Donald Trump"]

pres16 = pres16[['pct']]

pres16.dropna(inplace=True)

pres16.head(10)

pct
County State          
NaN    US     0.472993
       CA     0.330641
       FL     0.490640
       TX     0.525830
       NY     0.374752
       PA     0.487921
       IL     0.394133
       OH     0.520522
       MI     0.475881
       NC     0.505372

In [11]:
all_together = county_2016.merge(pres16, on=["County", "State"])
all_together.dropna(inplace=True)
all_together.drop("Year", axis=1, inplace=True) # removing year column (not row) since we know it is 2016,
all_together.tail(10)

,,Month,Rate,min_wage,pct
County,State,,,,
Washington County,ME,February,7.8,7.82,0.545746
York County,ME,February,4.1,7.82,0.442147
Waldo County,ME,February,5.2,7.82,0.458595
Penobscot County,ME,February,4.9,7.82,0.518535
Franklin County,ME,February,4.8,7.82,0.482237
Somerset County,ME,February,6.6,7.82,0.577557
Oxford County,ME,February,5.2,7.82,0.521451
Knox County,ME,February,4.7,7.82,0.397376
Piscataquis County,ME,February,5.7,7.82,0.590879


In [12]:
all_together.corr() # interesting negative corrleation between minimum wage and percent of people who voted for Trump
# as minimum wage riss, the amount of people who voted for Trump decreases and vice-versa

,Rate,min_wage,pct
Rate,1.000000,0.043354,0.072490
min_wage,0.043354,1.000000,-0.302582
pct,0.072490,-0.302582,1.000000


In [13]:
all_together.cov() # minimum wage and percent voted for Trump don't seem to have shared covariance though.
# important to note that correlation is standarised though versus covariance that is not, an inverse relationship
# could really exist 


,Rate,min_wage,pct
Rate,5.901587,0.172792,0.026739
min_wage,0.172792,2.691630,-0.075377
pct,0.026739,-0.075377,0.023056
